# Przetwarzanie wstępne. Filtracja kontekstowa.


### Cel:
- zapoznanie z pojęciem kontekstu / filtracji kontekstowej,
- zapoznanie z pojęciem konwolucji (splotu),
- zapoznanie z wybranymi filtrami:
	- filtry liniowe dolnoprzepustowe:
		- filtr uśredniający,
		- filtr Gaussa.
	- filtry nielinowe:
		- mediana,
		- mediana dla obrazów kolorowych.
	- filtry liniowe górnoprzepustowe:
			- laplasjan,
			- operator Robersta, Prewitta, Sobela.
- zadanie domowe: adaptacyjna filtracja medianowa.

### Filtry liniowe uśredniające (dolnoprzepustowe)

Jest to podstawowa rodzina filtrów stosowana w cyfrowym przetwarzaniu obrazów. 
Wykorzystuje się je w celu "rozmazania" obrazu i tym samym redukcji szumów (zakłóceń) na obrazie.
Filtr określony jest przez dwa parametry: rozmiar maski (ang. _kernel_) oraz wartości współczynników maski.

Warto zwrócić uwagę, że omawiane w niniejszym rozdziale operacje generują nową wartość piksela na podstawie pewnego fragmentu obrazu (tj. kontekstu), a nie jak operacje punktowe tylko na podstawie jednego piksela.


1. Wczytaj obraz _plansza.png_.
W dalszej części ćwiczenia sprawdzenie działania filtracji dla innych obrazów sprowadzi się do wczytania innego pliku.

2. Podstawowa funkcja to `cv2.filter2D`  - realizacja filtracji konwolucyjnej.
   Proszę sprawdzić jej dokumentację i zwrócić uwagę na obsługę problemu brzegowego (na krawędziach istnieją piksele dla których nie da się wyznaczyć otoczenia).

  Uwaga. Problem ten można też rozwiązać z użyciem funkcji `signal.convolve2d` z biblioteki _scipy_ (`from scipy import signal`).

3. Stwórz podstawowy filtr uśredniający o rozmiarze $3 \times 3$ -- za pomocą funkcji `np.ones`. Wykonaj konwolucję na wczytanym obrazie. Na wspólnym rysunku wyświetl obraz oryginalny, po filtracji oraz moduł z różnicy.

4. Przeanalizuj otrzymane wyniki. Jakie elementy zawiera obraz "moduł z różnicy"? Co na tej podstawie można powiedzieć o filtracji dolnoprzepustowej?

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np
from scipy import signal

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/'

fileNames = ["jet.png", "kw.png", "moon.png", "lenaSzum.png", "lena.png", "plansza.png"]
for fileName in fileNames:
  if not os.path.exists(fileName):
      r = requests.get(url + fileName, allow_redirects=True)
      open(fileName, 'wb').write(r.content)

img_plansza = cv2.imread("plansza.png")
img_plansza = cv2.cvtColor(img_plansza, cv2.COLOR_BGR2GRAY)

kernel_ones = np.ones((3,3))

fig, axs = plt.subplots(1,3)
fig.set_size_inches(15, 8)
axs[0].axis('off')
axs[1].axis('off')
axs[2].axis('off')
axs[0].imshow(img_plansza, 'gray')
img_plansza_filtered = (signal.convolve2d(img_plansza, kernel_ones, mode='same', boundary='symm')/9).astype(int)
print(type(img_plansza))
print(type(img_plansza_filtered))
axs[1].imshow(img_plansza_filtered, 'gray')
axs[2].imshow(abs(np.subtract(img_plansza, img_plansza_filtered)), 'gray')

5. Na wspólnym rysunku wyświetl wyniki filtracji uśredniającej z oknem o rozmiarze 3, 5, 9, 15 i 35. 
Wykorzystaj polecenie `plt.subplot`. 
Przeanalizuj wpływ rozmiaru maski na wynik. 

In [ ]:
def apply_filtration(img, kernel_size):
    return (signal.convolve2d(img, np.ones((kernel_size, kernel_size)), mode='full', boundary='fill', fillvalue=0))/kernel_size

fig, axs = plt.subplots(1,5)
fig.set_size_inches(15, 8)
axs[0].axis('off')
axs[0].set_title('3x3')
axs[1].axis('off')
axs[1].set_title('5x5')
axs[2].axis('off')
axs[2].set_title('9x9')
axs[3].axis('off')
axs[3].set_title('15x15')
axs[4].axis('off')
axs[4].set_title('35x35')
axs[0].imshow(apply_filtration(img_plansza, 3), 'gray')
axs[1].imshow(apply_filtration(img_plansza, 5), 'gray')
axs[2].imshow(apply_filtration(img_plansza, 9), 'gray')
axs[3].imshow(apply_filtration(img_plansza, 15), 'gray')
axs[4].imshow(apply_filtration(img_plansza, 35), 'gray')

6. Wczytaj obraz _lena.png_.
Zaobserwuj efekty filtracji dolnoprzepustowej dla obrazu rzeczywistego.

In [ ]:
img_lena = cv2.imread("lena.png")
img_lena = cv2.cvtColor(img_lena, cv2.COLOR_BGR2GRAY)

fig, axs = plt.subplots(1,4)
fig.set_size_inches(15, 8)
axs[0].axis('off')
axs[0].set_title('3x3')
axs[1].axis('off')
axs[1].set_title('5x5')
axs[2].axis('off')
axs[2].set_title('15x15')
axs[3].axis('off')
axs[3].set_title('30x30')
axs[0].imshow(apply_filtration(img_lena, 3), 'gray')
axs[1].imshow(apply_filtration(img_lena, 5), 'gray')
axs[2].imshow(apply_filtration(img_lena, 15), 'gray')
axs[3].imshow(apply_filtration(img_lena, 30), 'gray')

7. Niekorzystny efekt towarzyszący wykonanym filtracjom dolnoprzepustowym to utrata ostrości. 
Częściowo można go zniwelować poprzez odpowiedni dobór maski. 
Wykorzystaj maskę:  `M = np.array([1 2 1; 2 4 2; 1 2 1])`. 
Przed obliczeniami należy jeszcze wykonać normalizację - podzielić każdy element maski przez sumę wszystkich elementów: `M = M/sum(sum(M));`.
Tak przygotowaną maskę wykorzystaj w konwolucji - wyświetl wyniki tak jak wcześniej.
Możliwe jest też wykorzystywanie innych masek - współczynniki można dopasowywać do konkretnego problemu.

In [ ]:
kernel_M = np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]])
kernel_M = kernel_M/sum(sum(kernel_M))

fig, axs = plt.subplots(1,4)
fig.set_size_inches(15, 8)
axs[0].axis('off')
axs[0].set_title('Oryginał')
axs[1].axis('off')
axs[1].set_title('Po konwulucji')
axs[0].imshow(img_lena, 'gray')
axs[1].imshow((signal.convolve2d(img_lena, kernel_M, mode='full', boundary='fill', fillvalue=0)), 'gray')
axs[2].axis('off')
axs[2].set_title('Oryginał')
axs[3].axis('off')
axs[3].set_title('Po konwulucji')
axs[2].imshow(img_plansza, 'gray')
axs[3].imshow((signal.convolve2d(img_plansza, kernel_M, mode='full', boundary='fill', fillvalue=0)), 'gray')

8. Skuteczną i często wykorzystywaną maską jest tzw. maska Gasussa.
Jest to zbiór liczb, które aproksymują dwuwymiarowy rozkład Gaussa. 
Parametrem jest odchylenie standardowe i rozmiar maski.

9. Wykorzystując przygotowaną funkcję `fgaussian` stwórz maskę o rozmiarze $5 \times 5$ i odchyleniu standardowym 0.5.
  Wykorzystując funkcję `mesh` zwizualizuj filtr.
  Sprawdź jak parametr ``odchylenie standardowe'' wpływa na ``kształt'' filtru.

  Uwaga. W OpenCV dostępna jest *dedykowana* funkcja do filtracji Gaussa - `GaussianBlur`.
  Proszę na jednym przykładzie porównać jej działanie z użytym wyżej rozwiązaniem.

10. Wykonaj filtrację dla wybranych (2--3) wartości odchylenia standardowego.


In [ ]:
def fgaussian(size, sigma):
     m = n = size
     h, k = m//2, n//2
     x, y = np.mgrid[-h:h+1, -k:k+1]
     g = np.exp(-(x**2 + y**2)/(2*sigma**2))
     return g /g.sum() 
    
    
def mesh(fun, size, sigma):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')

    X = np.arange(-size//2, size//2, 1)
    Y = np.arange(-size//2, size//2, 1)
    X, Y = np.meshgrid(X, Y)
    Z = fun
    
    ax.plot_surface(X, Y, Z)
    plt.title(f'Size: {size} ; Sigma: {sigma}')
    plt.show()
    
mesh(fgaussian(5, 0.5), 5, 0.5)
mesh(fgaussian(5, 0.75), 5, 0.75)
mesh(fgaussian(5, 1), 5, 1)
mesh(fgaussian(11, 0.5), 11, 0.5)
mesh(fgaussian(11, 0.75), 11, 0.75)
mesh(fgaussian(11, 2), 11, 2)
mesh(cv2.getGaussianKernel(11, 2), 11, 2)

In [ ]:
def apply_gaussian_filter(img, kernel_size, sigma):
    return (signal.convolve2d(img, fgaussian(kernel_size, sigma), mode='full', boundary='fill', fillvalue=0))/kernel_size

fig, axs = plt.subplots(1,5)
fig.set_size_inches(15, 8)
axs[0].axis('off')
axs[0].set_title('Oryginał')
axs[1].axis('off')
axs[1].set_title('5x5, σ-0.5')
axs[2].axis('off')
axs[2].set_title('5x5, σ-0.75')
axs[3].axis('off')
axs[3].set_title('5x5, σ-0.9')
axs[4].axis('off')
axs[4].set_title('5x5, σ-0.9, dedykowana')
axs[0].imshow(img_lena, 'gray')
axs[1].imshow(apply_gaussian_filter(img_lena, 5, 0.5), 'gray')
axs[2].imshow(apply_gaussian_filter(img_lena, 5, 0.75), 'gray')
axs[3].imshow(apply_gaussian_filter(img_lena, 5, 0.9), 'gray')
axs[4].imshow(cv2.GaussianBlur(img_lena,(5,5),0.9), 'gray')

### Filtry nieliniowe -- mediana

Filtry rozmywające redukują szum, ale niekorzystnie wpływają na ostrość obrazu.
Dlatego często wykorzystuje się filtry nieliniowe - np. filtr medianowy (dla przypomnienia: mediana - środkowa wartość w posortowanym ciągu liczb).

Podstawowa różnica pomiędzy filtrami liniowymi, a nieliniowymi polega na tym, że przy filtracji liniowej na nową wartość piksela ma wpływ wartość wszystkich pikseli z otoczenia (np. uśrednianie, czasem ważone), natomiast w przypadku filtracji nieliniowej jako nowy piksel wybierana jest któraś z wartości otoczenia - według jakiegoś wskaźnika (wartość największa, najmniejsza czy właśnie mediana).


1. Wczytaj obraz _lenaSzum.png_ (losowe 10% pikseli białych lub czarnych - tzw. zakłócenia impulsowe). Przeprowadź filtrację uśredniającą z rozmiarem maski 3x3. Wyświetl, podobnie jak wcześniej, oryginał, wynik filtracji i moduł z różnicy. Wykorzystując funkcję ``cv2.medianBlur` wykonaj filtrację medianową _lenaSzum.png_ (z rozmiarem maski $3 \times 3$). Wyświetl, podobnie jak wcześniej, oryginał, wynik filtracji i moduł z różnicy. Która filtracja lepiej radzi sobie z tego typu szumem?

  Uwaga. Taki sam efekt da również użycie funkcji `signal.medfilt2d`.


In [ ]:
img_lena_szum = cv2.imread("lenaSzum.png")
img_lena_szum = cv2.cvtColor(img_lena_szum, cv2.COLOR_BGR2GRAY)

fig, axs = plt.subplots(1,3)
fig.set_size_inches(15, 8)
fig.suptitle('Filtracaja uśredniająca')
axs[0].axis('off')
axs[0].imshow(img_lena_szum, 'gray')
axs[0].set_title('Oryginał')
axs[1].axis('off')
axs[1].imshow(apply_filtration(img_lena_szum, 3), 'gray')
axs[1].set_title('Wynik filtracji')
axs[2].imshow(abs(np.subtract(img_lena_szum, (signal.convolve2d(img_lena_szum, kernel_ones, mode='same', boundary='symm')/9))), 'gray')
axs[2].axis('off')
axs[2].set_title('Różnica')

fig, axs = plt.subplots(1,3)
fig.set_size_inches(15, 8)
fig.suptitle('Filtracaja medianowa')
axs[0].axis('off')
axs[0].imshow(img_lena_szum, 'gray')
axs[0].set_title('Oryginał')
axs[1].axis('off')
axs[1].imshow(cv2.medianBlur(img_lena_szum, 3), 'gray')
axs[1].set_title('Wynik filtracji')
axs[2].imshow(abs(np.subtract(img_lena_szum, cv2.medianBlur(img_lena_szum, 3))), 'gray')
axs[2].axis('off')
axs[2].set_title('Różnica')

2. Przeprowadź filtrację uśredniającą, a następnie medianową obrazu _lena.png_.
   Wyniki porównaj - dla obu wyświetl: oryginał, wynik filtracji i moduł z różnicy.
   Szczególną uwagę zwróć na ostrość i krawędzie.
   W której filtracji krawędzie zostają lepiej zachowane?

In [ ]:
fig, axs = plt.subplots(1,3)
fig.set_size_inches(15, 8)
fig.suptitle('Filtracaja uśredniająca')
axs[0].axis('off')
axs[0].imshow(img_lena, 'gray')
axs[0].set_title('Oryginał')
axs[1].axis('off')
axs[1].imshow(apply_filtration(img_lena, 3), 'gray')
axs[1].set_title('Wynik filtracji')
axs[2].imshow(abs(np.subtract(img_lena, (signal.convolve2d(img_lena, kernel_ones, mode='same', boundary='symm')/9))), 'gray')
axs[2].axis('off')
axs[2].set_title('Różnica')

fig, axs = plt.subplots(1,3)
fig.set_size_inches(15, 8)
fig.suptitle('Filtracaja medianowa')
axs[0].axis('off')
axs[0].imshow(img_lena, 'gray')
axs[0].set_title('Oryginał')
axs[1].axis('off')
axs[1].imshow(cv2.medianBlur(img_lena, 3), 'gray')
axs[1].set_title('Wynik filtracji')
axs[2].imshow(abs(np.subtract(img_lena, cv2.medianBlur(img_lena, 3))), 'gray')
axs[2].axis('off')
axs[2].set_title('Różnica')

3. Ciekawy efekt można uzyskać wykonując filtrację medianową wielokrotnie. Określa się go mianem  posteryzacji.  W wyniku przetwarzania z obrazka usunięte zostają detale, a duże obszary uzyskują tą samą wartość jasności.  Wykonaj operację mediany $5 \times 5$ na obrazie _lena.png_ 10-krotnie. (wykorzystaj np. pętlę `for`).


Inne filtry nieliniowe:
- filtr modowy - moda (dominanta) zamiast mediany,
- filtr olimpijski - średnia z podzbioru otoczenia (bez wartości ekstremalnych),
- hybrydowy filtr medianowy - mediana obliczana osobno w różnych podzbiorach otoczenia (np. kształt ``x'',``+''), a jako wynik brana jest mediana ze zbioru wartość elementu centralnego, mediana z ``x'' i mediana z ``+'',
- filtr minimalny i maksymalny (będą omówione przy okazji operacji morfologicznych w dalszej części kursu).


Warto zdawać sobie sprawę, z szerokich możliwości dopasowywania rodzaju filtracji do konkretnego rozważanego problemu i rodzaju zaszumienia występującego na obrazie.

In [ ]:
img_lena_mult = cv2.imread("lena.png")
img_lena_mult = cv2.cvtColor(img_lena_mult, cv2.COLOR_BGR2GRAY)

for i in range(10):
    img_lena_mult = cv2.medianBlur(img_lena_mult, 5)

fig, axs = plt.subplots(1,3)
fig.set_size_inches(15, 8)
axs[0].axis('off')
axs[0].imshow(img_lena, 'gray')
axs[0].set_title('Oryginał')
axs[1].axis('off')
axs[1].imshow(img_lena_mult, 'gray')
axs[1].set_title('Wynik filtracji')
axs[2].imshow(abs(np.subtract(img_lena, img_lena_mult)), 'gray')
axs[2].axis('off')
axs[2].set_title('Różnica')

## Filtry liniowe górnoprzepustowe (wyostrzające, wykrywające krawędzie)

Zadaniem filtrów górnoprzepustowych jest wydobywanie z obrazu składników odpowiedzialnych za szybkie zmiany jasności - konturów, krawędzi, drobnych elementów tekstury.

### Laplasjan (wykorzystanie drugiej pochodnej obrazu)

1. Wczytaj obraz _moon.png_.

2. Wprowadź podstawową maskę laplasjanu:
\begin{equation}
M = 
\begin{bmatrix}
0 & 1& 0 \\ 1 & -4 & 1 \\ 0 & 1 & 0
\end{bmatrix}
\end{equation}

3. Przed rozpoczęciem obliczeń należy dokonać normalizacji maski - dla rozmiaru $3 \times 3$ podzielić każdy element przez 9.
   Proszę zwrócić uwagę, że nie można tu zastosować takiej samej normalizacji, jak dla filtrów dolnoprzepustowanych, gdyż skutkowałby to dzieleniem przez 0.

4. Wykonaj konwolucję obrazu z maską (`c2.filter2D`). Przed wyświetleniem, wynikowy obraz należy poddać normalizacji (występują ujemne wartości). Najczęściej wykonuje się jedną z dwóch operacji:
- skalowanie (np. poprzez dodatnie 128 do każdego z pikseli),
- moduł (wartość bezwzględna).

Wykonaj obie normalizacje. 
Na wspólnym wykresie wyświetl obraz oryginalny oraz przefiltrowany po obu normalizacjach. 

In [ ]:
img_moon = cv2.imread("moon.png")
img_moon = cv2.cvtColor(img_moon, cv2.COLOR_BGR2GRAY)

laplace = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])
# laplace = laplace/(laplace.size)

img_moon_scaled_filtered = cv2.filter2D(src=img_moon, ddepth=-1, kernel=laplace)+128
img_moon_abs_filtered = abs(cv2.filter2D(src=img_moon, ddepth=-1, kernel=laplace))

fig, axs = plt.subplots(1,3)
fig.set_size_inches(15, 8)
axs[0].axis('off')
axs[0].imshow(img_moon, 'gray')
axs[0].set_title('Oryginał')
axs[1].axis('off')
axs[1].imshow(img_moon_scaled_filtered, 'gray')
axs[1].set_title('Wynik filtracji po przeskalowaniu')
axs[2].imshow(img_moon_abs_filtered, 'gray')
axs[2].axis('off')
axs[2].set_title('Wynik filtracji po module')

7. Efekt wyostrzenia uzyskuje się po odjęciu/dodaniu (zależy do maski) rezultatu filtracji laplasjanowej i oryginalnego obrazu. Wyświetl na jednym wykresie: obraz oryginalny, sumę oryginału i wyniku filtracji oraz różnicę (bezwzględną) oryginału i wyniku filtracji.
 Uwaga. Aby uniknąć artefaktów, należy obraz wejściowy przekonwertować do formatu ze znakiem.



In [ ]:
fig, axs = plt.subplots(1,3)
fig.set_size_inches(15, 8)
fig.suptitle('Skalowanie', fontsize=20)
axs[0].axis('off')
axs[0].imshow(img_moon, 'gray')
axs[0].set_title('Oryginał')
axs[1].axis('off')
axs[1].imshow(np.add(img_moon.astype(np.int32), img_moon_scaled_filtered), 'gray')
axs[1].set_title('Dodanie obrazów')
axs[2].imshow(np.subtract(img_moon.astype(np.int32), img_moon_scaled_filtered), 'gray')
axs[2].axis('off')
axs[2].set_title('Odjęcie obrazów')

fig, axs = plt.subplots(1,3)
fig.set_size_inches(15, 8)
fig.suptitle('Moduł', fontsize=20)
axs[0].axis('off')
axs[0].imshow(img_moon, 'gray')
axs[0].set_title('Oryginał')
axs[1].axis('off')
axs[1].imshow(np.add(img_moon.astype(np.int32), img_moon_abs_filtered), 'gray')
axs[1].set_title('Dodanie obrazów')
axs[2].imshow(abs(np.subtract(img_moon.astype(np.int32), img_moon_abs_filtered)), 'gray')
axs[2].axis('off')
axs[2].set_title('Odjęcie obrazów')

### Gradienty (wykorzystanie pierwszej pochodnej obrazu)

1. Wczytaj obraz _kw.png_. Stwórz odpowiednie maski opisane w kolejnych punktach i dokonaj filtracji.
2. Wykorzystując gradient Robertsa przeprowadź detekcję krawędzi - poprzez wykonanie konwolucji obrazu z daną maską:
\begin{equation}
R1 = \begin{bmatrix} 0 & 0 & 0 \\ -1 & 0 & 0 \\ 0 & 1 & 0 \end{bmatrix}   
R2 = \begin{bmatrix} 0 & 0 & 0 \\ 0 & 0 & -1 \\ 0 & 1 & 0 \end{bmatrix}
\end{equation}

Wykorzystaj stworzony wcześniej kod (przy laplasjanie) - dwie metody normalizacji oraz sposób wyświetlania.

3. Analogicznie przeprowadź detekcję krawędzi za pomocą gradientu Prewitta (pionowy i poziomy)
\begin{equation}
P1 = \begin{bmatrix} -1 & 0 & 1 \\ -1 & 0 & 1 \\ -1 & 0 & 1 \end{bmatrix}   
P2 = \begin{bmatrix} -1 & -1 & -1 \\ 0 & 0 & 0 \\ 1 & 1 & 1 \end{bmatrix}
\end{equation}

4. Podobnie skonstruowany jest gradient Sobela (występuje osiem masek, zaprezentowane są dwie ``prostopadłe''):
\begin{equation}
S1 = \begin{bmatrix} -1 & 0 & 1 \\ -2 & 0 & 2 \\ -1 & 0 & 1 \end{bmatrix}   
S2 = \begin{bmatrix} -1 & -2 & -1 \\ 0 & 0 & 0 \\ 1 & 2 & 1 \end{bmatrix}
\end{equation}

Przeprowadź detekcję krawędzi za pomocą gradientu Sobela. 

In [ ]:
img_kw = cv2.imread("kw.png")
img_kw = cv2.cvtColor(img_kw, cv2.COLOR_BGR2GRAY)

r1 = np.array([[0, 0, 0], [-1, 0, 0], [0, 1, 0]])
r1 = r1/(r1.size)

r2 = np.array([[0, 0, 0], [0, 0, -1], [0, 1, 0]])
r2 = r2/(r2.size)

p1 = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
p1 = p1/(p1.size)

p2 = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])
p2 = p2/(p2.size)

s1 = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
s1 = s1/(s1.size)

s2 = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
s2 = s2/(s2.size)

fig, axs = plt.subplots(1,3)
fig.set_size_inches(15, 8)
axs[0].axis('off')
axs[0].imshow(img_kw, 'gray')
axs[0].set_title('Oryginał')
axs[1].axis('off')
axs[1].imshow(cv2.filter2D(src=img_kw, ddepth=-1, kernel=r1)+128, 'gray')
axs[1].set_title('R1')
axs[2].imshow(cv2.filter2D(src=img_kw, ddepth=-1, kernel=r2)+128, 'gray')
axs[2].axis('off')
axs[2].set_title('R2')

fig, axs = plt.subplots(1,3)
fig.set_size_inches(15, 8)
axs[0].axis('off')
axs[0].imshow(img_kw, 'gray')
axs[0].set_title('Oryginał')
axs[1].axis('off')
axs[1].imshow(cv2.filter2D(src=img_kw, ddepth=-1, kernel=p1), 'gray')
axs[1].set_title('P1')
axs[2].imshow(cv2.filter2D(src=img_kw, ddepth=-1, kernel=p2), 'gray')
axs[2].axis('off')
axs[2].set_title('P2')

fig, axs = plt.subplots(1,3)
fig.set_size_inches(15, 8)
axs[0].axis('off')
axs[0].imshow(img_kw, 'gray')
axs[0].set_title('Oryginał')
axs[1].axis('off')
axs[1].imshow(cv2.filter2D(src=img_kw, ddepth=-1, kernel=s1), 'gray')
axs[1].set_title('S1')
axs[2].imshow(cv2.filter2D(src=img_kw, ddepth=-1, kernel=s2), 'gray')
axs[2].axis('off')
axs[2].set_title('S2')

5. Na podstawie dwóch ortogonalnych masek np. Sobela można stworzyć tzw. filtr kombinowany - pierwiastek kwadratowy z sumy kwadratów gradientów:
\begin{equation}
OW = \sqrt{(O * S1)^2 + (O * S2)^2}
\end{equation}
gdzie:  $OW$ - obraz wyjściowy, $O$ - obraz oryginalny (wejściowy), $S1,S2$ - maski Sobela, $*$ - operacja konwolucji.

Zaimplementuj filtr kombinowany.

Uwaga. Proszę zwrócić uwagę na konieczność zmiany formatu danych obrazu wejściowego - na typ znakiem



In [ ]:
def combined_sqr(img):
    s1 = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    s1 = s1/(s1.size)

    s2 = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
    s2 = s2/(s2.size)
    
    S1 = abs(cv2.filter2D(src=img, ddepth=-1, kernel=s1))
    S2 = abs(cv2.filter2D(src=img, ddepth=-1, kernel=s2))
    
    return np.sqrt(np.square(S1.astype(np.int32)) + np.square(S2.astype(np.int32)))

fig, axs = plt.subplots(1,2)
fig.set_size_inches(15, 8)
axs[0].axis('off')
axs[0].imshow(img_kw, 'gray')
axs[0].set_title('Oryginał')
axs[1].axis('off')
axs[1].imshow(combined_sqr(img_kw), 'gray')
axs[1].set_title('Filtr kombinowany')

6. Istnieje alternatywna wersja filtra kombinowanego, która zamiast pierwiastka z sumy kwadratów wykorzystuje sumę modułów (prostsze obliczenia). 
Zaimplementuj tę wersję. 

In [ ]:
def combined_abs(img):
    s1 = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    s1 = s1/(s1.size)

    s2 = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
    s2 = s2/(s2.size)
    
    S1 = abs(cv2.filter2D(src=img, ddepth=-1, kernel=s1))
    S2 = abs(cv2.filter2D(src=img, ddepth=-1, kernel=s2))
    
    return np.sqrt(abs(S1.astype(np.int32)) + abs(S2.astype(np.int32)))

fig, axs = plt.subplots(1,2)
fig.set_size_inches(15, 8)
axs[0].axis('off')
axs[0].imshow(img_kw, 'gray')
axs[0].set_title('Oryginał')
axs[1].axis('off')
axs[1].imshow(combined_abs(img_kw), 'gray')
axs[1].set_title('Filtr kombinowany')

7. Wczytaj plik _jet.png_ (zamiast _kw.png_).
Sprawdź działanie obu wariantów filtracji kombinowanej.

In [ ]:
img_jet = cv2.imread("jet.png")
img_jet = cv2.cvtColor(img_jet, cv2.COLOR_BGR2GRAY)

fig, axs = plt.subplots(1,2)
fig.set_size_inches(15, 8)
axs[0].axis('off')
axs[0].imshow(img_jet, 'gray')
axs[0].set_title('Oryginał')
axs[1].axis('off')
axs[1].imshow(combined_sqr(img_jet), 'gray')
axs[1].set_title('Filtr kombinowany sqr')

fig, axs = plt.subplots(1,2)
fig.set_size_inches(15, 8)
axs[0].axis('off')
axs[0].imshow(img_jet, 'gray')
axs[0].set_title('Oryginał')
axs[1].axis('off')
axs[1].imshow(combined_abs(img_jet), 'gray')
axs[1].set_title('Filtr kombinowany abs')